# Cross validation via MLJ

In [1]:
using MLJ
import RDatasets: dataset
import DataFrames: DataFrame, select
auto = dataset("ISLR", "Auto")
y, X = unpack(auto, ==(:MPG), col->true)
train, test = partition(eachindex(y), 0.5, shuffle=true, rng=444);

In [2]:
LR = @load LinearRegressor pkg=MLJLinearModels

[ Info: For silent loading, specify `verbosity=0`. 


import MLJLinearModels ✔


MLJLinearModels.LinearRegressor

In [3]:
lm = LR()
mlm = machine(lm, select(X, :Horsepower), y) #Only use horsepower as predictor
fit!(mlm, rows=train)
rms(MLJ.predict(mlm, rows=test), y[test])^2

[ Info: Training machine(LinearRegressor(fit_intercept = true, …), …).
┌ Info: Solver: MLJLinearModels.Analytical
│   iterative: Bool false
└   max_inner: Int64 200


23.493990895007986

Create 3 models up third order poly by creating a df with each order:

In [4]:
hp = X.Horsepower
Xhp = DataFrame(hp1=hp, hp2=hp.^2, hp3=hp.^3);

Create a pipeline, for selecting the desired feature. Accessed like a struct.

In [5]:
LinModPipe = FeatureSelector(features=[:hp1]) |> LR()

DeterministicPipeline(
  feature_selector = FeatureSelector(
        features = [:hp1], 
        ignore = false), 
  linear_regressor = LinearRegressor(
        fit_intercept = true, 
        solver = nothing), 
  cache = true)

Create 3 models with different features:

In [6]:
lr1 = machine(LinModPipe, Xhp, y) # poly of degree 1 (line)
fit!(lr1, rows=train)

LinModPipe.feature_selector.features = [:hp1, :hp2] # poly of degree 2
lr2 = machine(LinModPipe, Xhp, y)
fit!(lr2, rows=train)

LinModPipe.feature_selector.features = [:hp1, :hp2, :hp3] # poly of degree 3
lr3 = machine(LinModPipe, Xhp, y)
fit!(lr3, rows=train)

[ Info: Training machine(DeterministicPipeline(feature_selector = FeatureSelector(features = [:hp1], …), …), …).
[ Info: Training machine(:feature_selector, …).
[ Info: Training machine(:linear_regressor, …).
┌ Info: Solver: MLJLinearModels.Analytical
│   iterative: Bool false
└   max_inner: Int64 200
[ Info: Training machine(DeterministicPipeline(feature_selector = FeatureSelector(features = [:hp1, :hp2], …), …), …).
[ Info: Training machine(:feature_selector, …).
[ Info: Training machine(:linear_regressor, …).
┌ Info: Solver: MLJLinearModels.Analytical
│   iterative: Bool false
└   max_inner: Int64 200
[ Info: Training machine(DeterministicPipeline(feature_selector = FeatureSelector(features = [:hp1, :hp2, :hp3], …), …), …).
[ Info: Training machine(:feature_selector, …).
[ Info: Training machine(:linear_regressor, …).
┌ Info: Solver: MLJLinearModels.Analytical
│   iterative: Bool false
└   max_inner: Int64 200


trained Machine; does not cache data
  model: DeterministicPipeline(feature_selector = FeatureSelector(features = [:hp1, :hp2, :hp3], …), …)
  args: 
    1:	Source @203 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @904 ⏎ AbstractVector{Continuous}


In [7]:
get_mse(lr) = rms(MLJ.predict(lr, rows=test), y[test])^2

@show get_mse(lr1)
@show get_mse(lr2)
@show get_mse(lr3)

get_mse(lr1) = 23.493990895007986
get_mse(lr2) = 19.287175510952164
get_mse(lr3) = 19.381831638657914


19.381831638657914

## Cross Validation

To create the sets of poly orders up to N, create an array of the sets ie 1, 1 2 etc then use a tuned model with cross validation to compare.

In [8]:
Xhp = DataFrame([hp.^i for i in 1:10], :auto)

cases = [[Symbol("x$j") for j in 1:i] for i in 1:10]
r = range(LinModPipe, :(feature_selector.features), values=cases)

tm = TunedModel(model=LinModPipe, ranges=r, resampling=CV(nfolds=10), measure=rms)
#This is a tuned model, not a trained machine.

DeterministicTunedModel(
  model = DeterministicPipeline(
        feature_selector = FeatureSelector(features = [:hp1, :hp2, :hp3], …), 
        linear_regressor = LinearRegressor(fit_intercept = true, …), 
        cache = true), 
  tuning = RandomSearch(
        bounded = Distributions.Uniform, 
        positive_unbounded = Distributions.Gamma, 
        other = Distributions.Normal, 
        rng = Random._GLOBAL_RNG()), 
  resampling = CV(
        nfolds = 10, 
        shuffle = false, 
        rng = Random._GLOBAL_RNG()), 
  measure = RootMeanSquaredError(), 
  weights = nothing, 
  class_weights = nothing, 
  operation = nothing, 
  range = NominalRange(feature_selector.features = [:x1], [:x1, :x2], [:x1, :x2, :x3], ...), 
  selection_heuristic = MLJTuning.NaiveSelection(nothing), 
  train_best = true, 
  repeats = 1, 
  n = nothing, 
  acceleration = CPU1{Nothing}(nothing), 
  acceleration_resampling = CPU1{Nothing}(nothing), 
  check_measure = true, 
  cache = true)

In [11]:
mtm = machine(tm, Xhp, y)
fit!(mtm)
rep = report(mtm)

rep.best_model

[ Info: Training machine(DeterministicTunedModel(model = DeterministicPipeline(feature_selector = FeatureSelector(features = [:hp1, :hp2, :hp3], …), …), …), …).
[ Info: Attempting to evaluate 10 models.
Evaluating over 10 metamodels: 100%[=========================] Time: 0:00:00


DeterministicPipeline(
  feature_selector = FeatureSelector(
        features = [:x1, :x2, :x3, :x4, :x5], 
        ignore = false), 
  linear_regressor = LinearRegressor(
        fit_intercept = true, 
        solver = nothing), 
  cache = true)

In [10]:
res = rep.plotting

@show round.(res.measurements.^2, digits=2)
@show argmin(res.measurements)

round.(res.measurements .^ 2, digits = 2) = [21.35, 27.44, 21.24, 20.91, 20.91, 25.66, 21.35, 21.35, 88.61, 88.61]
argmin(res.measurements) = 4


4